In [1]:
# Importando as bibliotecas
from CoolProp.CoolProp import PropsSI, PhaseSI
import pprint
import pandas as pd
import matplotlib.pyplot as plt
import copy

In [2]:
# Calcula qualquer ponto informando dicionario com duas variaveis
def calculate_point(point):
    variables = ['T', 'H', 'S', 'Q', 'P']
    input_variables = list(point.keys())
    output_variables = [variable for variable in variables if variable not in input_variables]
    output_values = PropsSI(output_variables, 
                            input_variables[0], 
                            point[input_variables[0]], 
                            input_variables[1], 
                            point[input_variables[1]], 
                            point['refrigerant']
    )

    for index, variable in enumerate(output_variables):
        point[variable] = output_values[index]

    return point

In [3]:
# Calcula ciclo baseado nas variaveis de entrada
def calculate_cycle(cycle_inputs):

    # Point 3 (after condenser)
    point_3_saturado = {'Q': 0, 
                        'T': cycle_inputs['t_external_env'] + cycle_inputs['approach_condenser'],
                       'refrigerant': cycle_inputs['refrigerant']
                       }
    calculate_point(point_3_saturado)

    if cycle_inputs['subcooling'] == 0:
        point_3 = point_3_saturado
    else:
        point_3 = {'P': point_3_saturado['P'], 
                   'T': point_3_saturado['T'] - cycle_inputs['subcooling'],
                   'refrigerant': cycle_inputs['refrigerant']
                  }
        calculate_point(point_3)
        
    
    # Point 5 (after high temperature evaporator)    
    point_5_saturado = {'Q': 1, 
                        'T': cycle_inputs['t_internal_env_ht'] - cycle_inputs['approach_evaporator_ht'],
                        'refrigerant': cycle_inputs['refrigerant']
                       }
    calculate_point(point_5_saturado)
    
    if cycle_inputs['superheating_ht'] == 0:
        point_5 = point_5_saturado
    else:
        point_5 = {'P': point_5_saturado['P'], 
                   'T': point_5_saturado['T'] + cycle_inputs['superheating_ht'],
                   'refrigerant': cycle_inputs['refrigerant']
                  }
        calculate_point(point_5)

    # Point 8 (after low temperature evaporator)  
    point_8_saturado = {'Q': 1, 
                        'T': cycle_inputs['t_internal_env_lt'] - cycle_inputs['approach_evaporator_lt'],
                        'refrigerant': cycle_inputs['refrigerant']
                       }
    calculate_point(point_8_saturado)

    if cycle_inputs['superheating_lt'] == 0:
        point_8 = point_8_saturado
    else:
        point_8 = {'P': point_8_saturado['P'], 
                   'T': point_8_saturado['T'] + cycle_inputs['superheating_lt'],
                   'refrigerant': cycle_inputs['refrigerant']
                  }
        calculate_point(point_8)        
    
    # Point 4 (after expansion valve of HTE)  
    point_4 = {'P': point_5['P'], 
               'H': point_3['H'],
               'refrigerant': cycle_inputs['refrigerant']
              }
    calculate_point(point_4)

    # Point 7 (after expansion valve of LTE)  
    point_7 = {'P': point_8['P'], 
               'H': point_3['H'],
               'refrigerant': cycle_inputs['refrigerant']
              }
    calculate_point(point_7)
    
    # Point 6 (after regulator expansion valve)  
    point_6 = {'P': point_8['P'], 
               'H': point_5['H'],
               'refrigerant': cycle_inputs['refrigerant']
              }
    calculate_point(point_6)
    
    # Point 1 (before compressor)
    point_1 = {'P': point_8['P'],
               'H': (1 - cycle_inputs['f']) * point_6['H'] + cycle_inputs['f'] * point_8['H'],
               'refrigerant': cycle_inputs['refrigerant']
    }
    calculate_point(point_1)
    
    # Point 2 (after compressor)
    point_2_isen = {'S': point_1['S'], 
                    'P': point_3['P'],
                    'refrigerant': cycle_inputs['refrigerant']
                   }
    calculate_point(point_2_isen)

    point_2 = {'P': point_3['P'], 
               'H': point_1['H'] + (point_2_isen['H'] - point_1['H']) / cycle_inputs['isentropic_efficiency'],
               'refrigerant': cycle_inputs['refrigerant']
              }
    calculate_point(point_2)
    
    # Masses
    m = cycle_inputs['work'] / (point_2['H'] - point_1['H'])
    m_lt = m * cycle_inputs['f']
    m_ht = m * (1 - cycle_inputs['f'])
    
    # COP
    q_evaporator_ht = m_ht * (point_5['H'] - point_4['H'])
    q_evaporator_lt = m_lt * (point_8['H'] - point_7['H'])
    cop = (q_evaporator_ht + q_evaporator_lt) / cycle_inputs['work'] 
    
    # Exergy Destruction
    t_0 = cycle_inputs['t_external_env']
    
    compressor_exergy_destruction = m * t_0 * (point_2['S'] - point_1['S'])
    condenser_exergy_destruction = m * t_0 * (point_3['S'] - point_2['S'] + (point_2['H'] - point_3['H']) / cycle_inputs['t_external_env'])
    expansion_valve_ht_exergy_destruction = m_ht * t_0 * (point_4['S'] - point_3['S'])
    evaporator_ht_exergy_destruction = m_ht * t_0 * (point_5['S'] - point_4['S'] - (point_5['H'] - point_4['H']) / cycle_inputs['t_internal_env_ht'])
    regulator_expansion_valve_exergy_destruction = m_ht * t_0 * (point_6['S'] - point_5['S'])
    expansion_valve_lt_exergy_destruction = m_lt * t_0 * (point_7['S'] - point_3['S'])
    evaporator_lt_exergy_destruction = m_lt * t_0 * (point_8['S'] - point_7['S'] - (point_8['H'] - point_7['H']) / cycle_inputs['t_internal_env_lt'])
    
    total_exergy_destruction = compressor_exergy_destruction + condenser_exergy_destruction \
        + expansion_valve_ht_exergy_destruction + evaporator_ht_exergy_destruction + regulator_expansion_valve_exergy_destruction \
        + expansion_valve_lt_exergy_destruction + evaporator_lt_exergy_destruction
    
    return {
        'cycle_inputs': cycle_inputs,
        'point_1': point_1,
        'point_2': point_2,
        'point_3': point_3,
        'point_4': point_4,
        'point_5': point_5,
        'point_6': point_6,
        'point_7': point_7,
        'point_8': point_8,
        'm': m,
        'm_ht': m_ht,
        'm_lt': m_lt,
        'q_evaporator_ht': q_evaporator_ht,
        'q_evaporator_lt': q_evaporator_lt,
        'cop': cop,
        'exergy_efficiency': 1 - total_exergy_destruction / cycle_inputs['work']
    }

In [4]:
# Variaveis de entrada
input_values = {
    't_external_env': 32.22 + 273.15,
    't_internal_env_ht': 4.44 + 273.15,
    't_internal_env_lt': -17.78 + 273.15,
    'approach_condenser': 0,
    'approach_evaporator_ht': 0,
    'approach_evaporator_lt': 0,
    'work': 18299.697604861303,
    'f': 0.3387218287321152,
    'isentropic_efficiency': 0.7,
    'subcooling': 5,
    'superheating_ht': 5,
    'superheating_lt': 5,
    'refrigerant': 'NH3'
}

cycle = calculate_cycle(input_values)
pprint.pprint(cycle)

{'cop': 2.8852935791667793,
 'cycle_inputs': {'approach_condenser': 0,
                  'approach_evaporator_ht': 0,
                  'approach_evaporator_lt': 0,
                  'f': 0.3387218287321152,
                  'isentropic_efficiency': 0.7,
                  'refrigerant': 'NH3',
                  'subcooling': 5,
                  'superheating_ht': 5,
                  'superheating_lt': 5,
                  't_external_env': 305.37,
                  't_internal_env_ht': 277.59,
                  't_internal_env_lt': 255.36999999999998,
                  'work': 18299.697604861303},
 'exergy_efficiency': 0.3811938000719234,
 'm': 0.04621471474880705,
 'm_ht': 0.030560782054758068,
 'm_lt': 0.01565393269404898,
 'point_1': {'H': 1616393.3717983814,
             'P': 209507.0301798974,
             'Q': -1.0,
             'S': 6468.565716701363,
             'T': 267.99108690371645,
             'refrigerant': 'NH3'},
 'point_2': {'H': 2012364.60529677,
             'P'

In [5]:
# Variaveis de entrada
input_values = {
    't_external_env': 25 + 273.15,
    't_internal_env_ht': 5 + 273.15,
    't_internal_env_lt': 0 + 273.15,
    'approach_condenser': 5,
    'approach_evaporator_ht': 5,
    'approach_evaporator_lt': 5,
    'work': 900,
    'f': 0.9,
    'isentropic_efficiency': 0.7,
    'subcooling': 10,
    'superheating_ht': 10,
    'superheating_lt': 10,
    'refrigerant': 'R290'
}
# 4.864088950174341
cycle = calculate_cycle(input_values)
pprint.pprint(cycle)

{'cop': 4.865878662304383,
 'cycle_inputs': {'approach_condenser': 5,
                  'approach_evaporator_ht': 5,
                  'approach_evaporator_lt': 5,
                  'f': 0.9,
                  'isentropic_efficiency': 0.7,
                  'refrigerant': 'R290',
                  'subcooling': 10,
                  'superheating_ht': 10,
                  'superheating_lt': 10,
                  't_external_env': 298.15,
                  't_internal_env_ht': 278.15,
                  't_internal_env_lt': 273.15,
                  'work': 900},
 'exergy_efficiency': 0.4356999337669615,
 'm': 0.013062394982416637,
 'm_ht': 0.0013062394982416635,
 'm_lt': 0.011756155484174973,
 'point_1': {'H': 586933.9849754514,
             'P': 406037.0303970629,
             'Q': -1.0,
             'S': 2442.6393311522106,
             'T': 278.4989694598495,
             'refrigerant': 'R290'},
 'point_2': {'H': 655834.060437225,
             'P': 1078995.2268894755,
             '

In [4]:
def determine_if_reached_threshold(value, lower, upper):
    reached_threshold = 0
    if value <= lower:
        reached_threshold = 1
        value = lower
    elif value >= upper:
        reached_threshold = 1
        value = upper  
    return value, reached_threshold

def calculate_next_x(input_values, x, current_cycle, delta, alpha, lower_threshold, upper_threshold):
    current_x = input_values[x]
    input_values_x = copy.copy(input_values)
    input_values_x[x] += delta
    gradient_cycle_x = calculate_cycle(input_values_x)
    gradient_x = (gradient_cycle_x['cop'] - current_cycle['cop']) / delta
    next_x = current_x + alpha * gradient_x
    next_x, reached_threshold_x = determine_if_reached_threshold(next_x, lower_threshold, upper_threshold)
    return next_x, reached_threshold_x

In [7]:
def calculate_next_all(input_values, delta, alpha):
      
    current_cycle = calculate_cycle(input_values)

    # Next superheating_ht
    next_superheating_ht, reached_threshold_superheating_ht = calculate_next_x(input_values, 
                                                                               'superheating_ht', 
                                                                               current_cycle, 
                                                                               delta, 
                                                                               alpha, 
                                                                               input_values['lower_threshold'], 
                                                                               input_values['upper_threshold'])

    # Next superheating_lt
    next_superheating_lt, reached_threshold_superheating_lt = calculate_next_x(input_values, 
                                                                               'superheating_lt', 
                                                                               current_cycle, 
                                                                               delta, 
                                                                               alpha, 
                                                                               input_values['lower_threshold'], 
                                                                               input_values['upper_threshold'])

    # Next subcooling
    next_subcooling, reached_threshold_subcooling = calculate_next_x(input_values, 
                                                                     'subcooling', 
                                                                     current_cycle, 
                                                                     delta, 
                                                                     alpha, 
                                                                     input_values['lower_threshold'], 
                                                                     input_values['upper_threshold'])
   
    # Next f
    next_f, reached_threshold_f = calculate_next_x(input_values, 'f', current_cycle, delta, alpha, 0.1, 0.9)
        
    input_values['superheating_ht'] = next_superheating_ht
    input_values['superheating_lt'] = next_superheating_lt
    input_values['subcooling'] = next_subcooling
    input_values['f'] = next_f
    sum_of_threshold_reached =  reached_threshold_superheating_ht + reached_threshold_superheating_lt \
        + reached_threshold_subcooling + reached_threshold_f
    
    next_cycle = calculate_cycle(input_values)
#     print('#'*30)
#     print('f: ', next_cycle['cycle_inputs']['f'])
#     print('subcooling: ', next_cycle['cycle_inputs']['subcooling'])
#     print('superheating_ht: ', next_cycle['cycle_inputs']['superheating_ht'])
#     print('superheating_lt: ', next_cycle['cycle_inputs']['superheating_lt'])
#     print('cop: ', next_cycle['cop'])
#     print('exergy_efficiency: ', next_cycle['exergy_efficiency'])
#     print('#'*30)
    
    return sum_of_threshold_reached, current_cycle, next_cycle

def optimize_with_cop(input_values):
    n = 0
    error = 1
    while n < 4 and abs(error) >= 10**(-10):
        n, current_cycle, next_cycle = calculate_next_all(input_values, 10**(-8), 5)
        error = (next_cycle['cop'] - current_cycle['cop'])/((next_cycle['cop'] + current_cycle['cop'])/2)
    optimized_cycle = calculate_cycle(input_values)
#     print('#'*30)
#     print('f: ', optimized_cycle['cycle_inputs']['f'])
#     print('subcooling: ', optimized_cycle['cycle_inputs']['subcooling'])
#     print('superheating_ht: ', optimized_cycle['cycle_inputs']['superheating_ht'])
#     print('superheating_lt: ', optimized_cycle['cycle_inputs']['superheating_lt'])
#     print('cop: ', optimized_cycle['cop'])
#     print('exergy_efficiency: ', optimized_cycle['exergy_efficiency'])
#     print('error: ', abs(error))
#     print('#'*30)
    return optimized_cycle

In [8]:
# Variaveis de entrada para otimizacao
input_values = {
    't_external_env': 25 + 273.15,
    't_internal_env_ht': 5 + 273.15,
    't_internal_env_lt': 0 + 273.15,
    'approach_condenser': 5,
    'approach_evaporator_ht': 5,
    'approach_evaporator_lt': 5,
    'work': 900,
    'f': 0.5,
    'isentropic_efficiency': 0.7,
    'subcooling': 5,
    'superheating_ht': 5,
    'superheating_lt': 5,
    'upper_threshold': 10,
    'lower_threshold': 0.1,
    'refrigerant': 'R290'
}
    
optimize_with_cop(input_values)

{'cycle_inputs': {'t_external_env': 298.15,
  't_internal_env_ht': 278.15,
  't_internal_env_lt': 273.15,
  'approach_condenser': 5,
  'approach_evaporator_ht': 5,
  'approach_evaporator_lt': 5,
  'work': 900,
  'f': 0.1,
  'isentropic_efficiency': 0.7,
  'subcooling': 10,
  'superheating_ht': 10,
  'superheating_lt': 10,
  'upper_threshold': 10,
  'lower_threshold': 0.1,
  'refrigerant': 'R290'},
 'point_1': {'P': 406037.0303970629,
  'H': 591715.4444424146,
  'refrigerant': 'R290',
  'T': 281.285318279968,
  'S': 2459.7226440621057,
  'Q': -1.0},
 'point_2': {'P': 1078995.2268894755,
  'H': 661566.5052464578,
  'refrigerant': 'R290',
  'T': 330.4620906956255,
  'S': 2524.1408916363507,
  'Q': -1.0},
 'point_3': {'P': 1078995.2268894755,
  'T': 293.15,
  'refrigerant': 'R290',
  'H': 251674.57795484507,
  'S': 1178.3627810313394,
  'Q': -1.0},
 'point_4': {'P': 474457.5428095771,
  'H': 251674.57795484507,
  'refrigerant': 'R290',
  'T': 273.15000000024503,
  'S': 1189.1802146649081,


In [8]:
def optimize_multiple_all_with_cop(input_values, input_ranges):
    original_input_values = copy.copy(input_values)
    results = pd.DataFrame(columns=['refrigerant',
                                    't_external_env',
                                    't_internal_env_ht',
                                    't_internal_env_lt',
                                    'f', 
                                    'subcooling', 
                                    'superheating_ht', 
                                    'superheating_lt', 
                                    'cop', 
                                    'exergy_efficiency'])
    n = 0
    for t_external_env in input_ranges['t_external_env_range']:
        t_external_env += 273.15
        for t_internal_env_ht in input_ranges['t_internal_env_ht_range']:
            t_internal_env_ht += 273.15
            for t_internal_env_lt in input_ranges['t_internal_env_lt_range']:
                t_internal_env_lt += 273.15
                for refrigerant in input_ranges['refrigerants']:
                    n += 1
                    print(n)
                    input_values = copy.copy(original_input_values)
                    input_values['t_external_env'] = t_external_env
                    input_values['t_internal_env_ht'] = t_internal_env_ht
                    input_values['t_internal_env_lt'] = t_internal_env_lt
                    input_values['refrigerant'] = refrigerant
                    optimized_cycle = optimize_with_cop(input_values)
                    results = results.append({
                        'refrigerant': refrigerant,
                        't_external_env': t_external_env - 273.15,
                        't_internal_env_ht': t_internal_env_ht - 273.15,
                        't_internal_env_lt': t_internal_env_lt - 273.15,
                        'f': optimized_cycle['cycle_inputs']['f'],
                        'subcooling': optimized_cycle['cycle_inputs']['subcooling'],
                        'superheating_ht': optimized_cycle['cycle_inputs']['superheating_ht'],
                        'superheating_lt': optimized_cycle['cycle_inputs']['superheating_lt'],
                        'cop': optimized_cycle['cop'],
                        'exergy_efficiency': optimized_cycle['exergy_efficiency']
                    }, ignore_index=True)
    return results

In [11]:
input_values = {
    'approach_condenser': 5,
    'approach_evaporator_ht': 5,
    'approach_evaporator_lt': 5,
    'work': 900,
    'f': 0.5,
    'isentropic_efficiency': 0.7,
    'subcooling': 5,
    'superheating_ht': 5,
    'superheating_lt': 5,
    'upper_threshold': 10,
    'lower_threshold': 0.1
}

input_ranges = {
#     't_external_env_range': [25, 30, 35, 40],
#     't_internal_env_ht_range': [5, 10, 15, 20],
#     't_internal_env_lt_range': [-15, -10, -5, 0],
#     'refrigerants': ['R134a', 'R22', 'R290', 'R600a', 'R1234yf', 'NH3']
    't_external_env_range': [25],
    't_internal_env_ht_range': [5],
    't_internal_env_lt_range': [-0.4, -0.3, -0.2, -0.1, 0.1, 0.2, 0.3, 0.4],
    'refrigerants': ['R134a']
}

optimized_table = optimize_multiple_all_with_cop(input_values, input_ranges)
optimized_table.to_excel(r'C:\repositories\projeto-final\src\very_low_error_r134a_optimized_table.xlsx', index = False)

1
2
3
4
5
6
7
8


In [30]:
def calculate_next_approaches(input_values, delta, alpha):
      
    current_cycle = calculate_cycle(input_values)

    next_approach_condenser, reached_threshold_approach_condenser = calculate_next_x(input_values, 
                                                                                     'approach_condenser', 
                                                                                     current_cycle, 
                                                                                     delta, 
                                                                                     alpha, 
                                                                                     input_values['lower_threshold'], 
                                                                                     input_values['upper_threshold'])

    next_approach_evaporator_ht, reached_threshold_approach_evaporator_ht = calculate_next_x(input_values, 
                                                                                             'approach_evaporator_ht', 
                                                                                             current_cycle, 
                                                                                             delta, 
                                                                                             alpha, 
                                                                                             input_values['lower_threshold'], 
                                                                                             input_values['upper_threshold'])
    
    next_approach_evaporator_lt, reached_threshold_approach_evaporator_lt = calculate_next_x(input_values, 
                                                                                             'approach_evaporator_lt', 
                                                                                             current_cycle, 
                                                                                             delta, 
                                                                                             alpha, 
                                                                                             input_values['lower_threshold'], 
                                                                                             input_values['upper_threshold'])
   
    input_values['approach_condenser'] = next_approach_condenser
    input_values['approach_evaporator_ht'] = next_approach_evaporator_ht
    input_values['approach_evaporator_lt'] = next_approach_evaporator_lt
    sum_of_threshold_reached =  reached_threshold_approach_condenser + reached_threshold_approach_evaporator_ht \
        + reached_threshold_approach_evaporator_lt
    
    next_cycle = calculate_cycle(input_values)
    
    return sum_of_threshold_reached, current_cycle, next_cycle

def optimize_approaches_with_cop(input_values):
    n = 0
    error = 1
    while n < 3 and abs(error) >= 10**(-10):
        n, current_cycle, next_cycle = calculate_next_approaches(input_values, 10**(-8), 5)
        error = (next_cycle['cop'] - current_cycle['cop'])/((next_cycle['cop'] + current_cycle['cop'])/2)
    optimized_cycle = calculate_cycle(input_values)
#     print('#'*30)
#     print('approach_condenser: ', optimized_cycle['cycle_inputs']['approach_condenser'])
#     print('approach_evaporator_ht: ', optimized_cycle['cycle_inputs']['approach_evaporator_ht'])
#     print('approach_evaporator_lt: ', optimized_cycle['cycle_inputs']['approach_evaporator_lt'])
#     print('cop: ', optimized_cycle['cop'])
#     print('exergy_efficiency: ', optimized_cycle['exergy_efficiency'])
#     print('#'*30)
    return optimized_cycle

In [31]:
# Variaveis de entrada para otimizacao
input_values = {
    't_external_env': 25 + 273.15,
    't_internal_env_ht': 5 + 273.15,
    't_internal_env_lt': 0 + 273.15,
    'approach_condenser': 5,
    'approach_evaporator_ht': 5,
    'approach_evaporator_lt': 5,
    'work': 900,
    'f': 0.5,
    'isentropic_efficiency': 0.7,
    'subcooling': 0,
    'superheating_ht': 0,
    'superheating_lt': 0,
    'upper_threshold': 10,
    'lower_threshold': 0,
    'refrigerant': 'R134a'
}
    
optimize_approaches_with_cop(input_values)

{'cycle_inputs': {'t_external_env': 298.15,
  't_internal_env_ht': 278.15,
  't_internal_env_lt': 273.15,
  'approach_condenser': 0,
  'approach_evaporator_ht': 0,
  'approach_evaporator_lt': 0,
  'work': 900,
  'f': 0.5,
  'isentropic_efficiency': 0.7,
  'subcooling': 0,
  'superheating_ht': 0,
  'superheating_lt': 0,
  'upper_threshold': 10,
  'lower_threshold': 0,
  'refrigerant': 'R134a'},
 'point_1': {'P': 292803.18233949516,
  'H': 400047.87204810267,
  'refrigerant': 'R134a',
  'T': 274.762428819819,
  'S': 1732.3582252482693,
  'Q': -1.0},
 'point_2': {'P': 665380.9325685131,
  'H': 424525.13355990796,
  'refrigerant': 'R134a',
  'T': 310.22392963676805,
  'S': 1756.3136144875887,
  'Q': -1.0},
 'point_3': {'Q': 0,
  'T': 298.15,
  'refrigerant': 'R134a',
  'H': 234545.77669487134,
  'S': 1119.9211777214425,
  'P': 665380.9325685131},
 'point_4': {'P': 349658.60786131525,
  'H': 234545.77669487134,
  'refrigerant': 'R134a',
  'T': 278.14999999999964,
  'S': 1124.2585710464475,


In [13]:
def optimize_multiple_approaches_with_cop(input_values, input_ranges):
    original_input_values = copy.copy(input_values)
    results = pd.DataFrame(columns=['refrigerant',
                                    't_external_env',
                                    't_internal_env_ht',
                                    't_internal_env_lt',
                                    'approach_condenser', 
                                    'approach_evaporator_ht', 
                                    'approach_evaporator_lt', 
                                    'cop', 
                                    'exergy_efficiency'])
    n = 0
    for t_external_env in input_ranges['t_external_env_range']:
        t_external_env += 273.15
        for t_internal_env_ht in input_ranges['t_internal_env_ht_range']:
            t_internal_env_ht += 273.15
            for t_internal_env_lt in input_ranges['t_internal_env_lt_range']:
                t_internal_env_lt += 273.15
                for refrigerant in input_ranges['refrigerants']:
                    n += 1
                    print(n)
                    input_values = copy.copy(original_input_values)
                    input_values['t_external_env'] = t_external_env
                    input_values['t_internal_env_ht'] = t_internal_env_ht
                    input_values['t_internal_env_lt'] = t_internal_env_lt
                    input_values['refrigerant'] = refrigerant
                    optimized_cycle = optimize_approaches_with_cop(input_values)
                    results = results.append({
                        'refrigerant': refrigerant,
                        't_external_env': t_external_env - 273.15,
                        't_internal_env_ht': t_internal_env_ht - 273.15,
                        't_internal_env_lt': t_internal_env_lt - 273.15,
                        'approach_condenser': optimized_cycle['cycle_inputs']['approach_condenser'],
                        'approach_evaporator_ht': optimized_cycle['cycle_inputs']['approach_evaporator_ht'],
                        'approach_evaporator_lt': optimized_cycle['cycle_inputs']['approach_evaporator_lt'],
                        'cop': optimized_cycle['cop'],
                        'exergy_efficiency': optimized_cycle['exergy_efficiency']
                    }, ignore_index=True)
    return results

In [14]:
input_values = {
    'approach_condenser': 5,
    'approach_evaporator_ht': 5,
    'approach_evaporator_lt': 5,
    'work': 900,
    'f': 0.5,
    'isentropic_efficiency': 0.7,
    'subcooling': 0,
    'superheating_ht': 0,
    'superheating_lt': 0,
    'upper_threshold': 10,
    'lower_threshold': 0
}

input_ranges = {
    't_external_env_range': [25, 30, 35, 40],
    't_internal_env_ht_range': [5, 10, 15, 20],
    't_internal_env_lt_range': [-15, -10, -5, 0],
    'refrigerants': ['R134a', 'R22', 'R290', 'R600a', 'R1234yf', 'NH3']
}

optimized_approaches_table = optimize_multiple_approaches_with_cop(input_values, input_ranges)
optimized_approaches_table.to_excel(r'C:\repositories\projeto-final\src\optimized_approaches_table.xlsx', index = False)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [15]:
def calculate_next_temperatures(input_values, delta, alpha):
      
    current_cycle = calculate_cycle(input_values)

    next_t_external_env, reached_threshold_t_external_env = calculate_next_x(input_values, 
                                                                             't_external_env', 
                                                                             current_cycle, 
                                                                             delta, 
                                                                             alpha, 
                                                                             30 + 273.15, 
                                                                             45 + 273.15)

    next_t_internal_env_ht, reached_threshold_t_internal_env_ht = calculate_next_x(input_values, 
                                                                                   't_internal_env_ht', 
                                                                                   current_cycle, 
                                                                                   delta, 
                                                                                   alpha, 
                                                                                   10 + 273.15, 
                                                                                   25 + 273.15)
    
    next_t_internal_env_lt, reached_threshold_t_internal_env_lt = calculate_next_x(input_values, 
                                                                                   't_internal_env_lt', 
                                                                                   current_cycle, 
                                                                                   delta, 
                                                                                   alpha, 
                                                                                   -10 + 273.15, 
                                                                                   5 + 273.15)
   
    input_values['t_external_env'] = next_t_external_env
    input_values['t_internal_env_ht'] = next_t_internal_env_ht
    input_values['t_internal_env_lt'] = next_t_internal_env_lt
    sum_of_threshold_reached =  reached_threshold_t_external_env + reached_threshold_t_internal_env_ht \
        + reached_threshold_t_internal_env_lt
    
    next_cycle = calculate_cycle(input_values)
    return sum_of_threshold_reached, current_cycle, next_cycle

def optimize_temperatures_with_cop(input_values):
    n = 0
    error = 1
    while n < 3 and abs(error) >= 10**(-8):
        n, current_cycle, next_cycle = calculate_next_temperatures(input_values, 10**(-8), 5)
        error = (next_cycle['cop'] - current_cycle['cop'])/((next_cycle['cop'] + current_cycle['cop'])/2)
    optimized_cycle = calculate_cycle(input_values)
#     print('#'*30)
#     print('t_external_env: ', optimized_cycle['cycle_inputs']['t_external_env'] - 273.15)
#     print('t_internal_env_ht: ', optimized_cycle['cycle_inputs']['t_internal_env_ht'] - 273.15)
#     print('t_internal_env_lt: ', optimized_cycle['cycle_inputs']['t_internal_env_lt'] - 273.15)
#     print('cop: ', optimized_cycle['cop'])
#     print('exergy_efficiency: ', optimized_cycle['exergy_efficiency'])
#     print('#'*30)
    return optimized_cycle

In [16]:
# Variaveis de entrada para otimizacao
input_values = {
    't_external_env': 35 + 273.15,
    't_internal_env_ht': 15 + 273.15,
    't_internal_env_lt': -5 + 273.15,
    'approach_condenser': 5,
    'approach_evaporator_ht': 5,
    'approach_evaporator_lt': 5,
    'work': 900,
    'f': 0.5,
    'isentropic_efficiency': 0.7,
    'subcooling': 0,
    'superheating_ht': 0,
    'superheating_lt': 0,
    'refrigerant': 'R134a'
}
    
optimize_temperatures_with_cop(input_values)

{'cycle_inputs': {'t_external_env': 303.15,
  't_internal_env_ht': 298.15,
  't_internal_env_lt': 278.15,
  'approach_condenser': 5,
  'approach_evaporator_ht': 5,
  'approach_evaporator_lt': 5,
  'work': 900,
  'f': 0.5,
  'isentropic_efficiency': 0.7,
  'subcooling': 0,
  'superheating_ht': 0,
  'superheating_lt': 0,
  'refrigerant': 'R134a'},
 'point_1': {'P': 292803.18233949516,
  'H': 404175.8872532415,
  'refrigerant': 'R134a',
  'T': 279.3921727807691,
  'S': 1747.2571136459046,
  'Q': -1.0},
 'point_2': {'P': 886980.9836083521,
  'H': 438183.572829408,
  'refrigerant': 'R134a',
  'T': 327.98625225142587,
  'S': 1778.838549059064,
  'Q': -1.0},
 'point_3': {'Q': 0,
  'T': 308.15,
  'refrigerant': 'R134a',
  'H': 249006.65185551936,
  'S': 1167.007935444416,
  'P': 886980.9836083521},
 'point_4': {'P': 571706.9090444488,
  'H': 249006.65185551936,
  'refrigerant': 'R134a',
  'T': 293.149999999998,
  'S': 1169.7150395378078,
  'Q': 0.11816354511214115},
 'point_5': {'Q': 1,
  'T':

In [17]:
def optimize_multiple_temperatures_with_cop(input_values, input_ranges):
    original_input_values = copy.copy(input_values)
    results = pd.DataFrame(columns=['refrigerant',
                                    't_external_env',
                                    't_internal_env_ht',
                                    't_internal_env_lt',
                                    'cop', 
                                    'exergy_efficiency'])
    n = 0
    for refrigerant in input_ranges['refrigerants']:
        n += 1
        print(n)
        input_values = copy.copy(original_input_values)
        input_values['refrigerant'] = refrigerant
        optimized_cycle = optimize_temperatures_with_cop(input_values)
        results = results.append({
            'refrigerant': refrigerant,
            't_external_env': optimized_cycle['cycle_inputs']['t_external_env'] - 273.15,
            't_internal_env_ht': optimized_cycle['cycle_inputs']['t_internal_env_ht'] - 273.15,
            't_internal_env_lt': optimized_cycle['cycle_inputs']['t_internal_env_lt'] - 273.15,
            'cop': optimized_cycle['cop'],
            'exergy_efficiency': optimized_cycle['exergy_efficiency']
        }, ignore_index=True)
    return results

In [18]:
input_values = {
    't_external_env': 35 + 273.15,
    't_internal_env_ht': 15 + 273.15,
    't_internal_env_lt': -5 + 273.15,
    'approach_condenser': 5,
    'approach_evaporator_ht': 5,
    'approach_evaporator_lt': 5,
    'work': 900,
    'f': 0.5,
    'isentropic_efficiency': 0.7,
    'subcooling': 0,
    'superheating_ht': 0,
    'superheating_lt': 0
}

input_ranges = {
    'refrigerants': ['R134a', 'R22', 'R290', 'R600a', 'R1234yf', 'NH3']
}

optimized_approaches_table = optimize_multiple_temperatures_with_cop(input_values, input_ranges)
optimized_approaches_table.to_excel(r'C:\repositories\projeto-final\src\optimized_temperatures_table.xlsx', index = False)

1
2
3
4
5
6


In [24]:
# Variaveis de entrada
input_values = {
    't_external_env': 25 + 273.15,
    't_internal_env_ht': 5 + 273.15,
    't_internal_env_lt': 0 + 273.15,
    'approach_condenser': 5,
    'approach_evaporator_ht': 5,
    'approach_evaporator_lt': 5,
    'work': 900,
    'f': 0.9,
    'isentropic_efficiency': 0.7,
    'subcooling': 10,
    'superheating_ht': 0.1,
    'superheating_lt': 0.1,
    'refrigerant': 'R134a'
}
# 4.924094886987869
cycle = calculate_cycle(input_values)
pprint.pprint(cycle)

{'cop': 4.924094886987869,
 'cycle_inputs': {'approach_condenser': 5,
                  'approach_evaporator_ht': 5,
                  'approach_evaporator_lt': 5,
                  'f': 0.9,
                  'isentropic_efficiency': 0.7,
                  'refrigerant': 'R134a',
                  'subcooling': 10,
                  'superheating_ht': 0.1,
                  'superheating_lt': 0.1,
                  't_external_env': 298.15,
                  't_internal_env_ht': 278.15,
                  't_internal_env_lt': 273.15,
                  'work': 900},
 'exergy_efficiency': 0.44091680802166433,
 'm': 0.026291961202324955,
 'm_ht': 0.002629196120232495,
 'm_lt': 0.02366276508209246,
 'point_1': {'H': 396040.9816873381,
             'P': 243342.36987785524,
             'Q': -1.0,
             'S': 1731.4560906402737,
             'T': 268.58685822611767,
             'refrigerant': 'R134a'},
 'point_2': {'H': 430271.97697423265,
             'P': 770196.3030768845,
        